In [4]:
# instalar versão atualizada
!pip install scipy==1.7.1
# instalar nova ferramenta para testes de hipoteses
!pip install pingouin

  Using cached scipy-1.7.1-cp39-cp39-win_amd64.whl (33.8 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.9.1
    Uninstalling scipy-1.9.1:
      Successfully uninstalled scipy-1.9.1


In [1]:
# gerais
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# analises
from scipy import stats as st
import pingouin as pg
# dados
dados_ab = pd.read_csv('exampleDataABtest.csv')
dados_nps = pd.read_csv('nps_example.csv', sep=';')

In [2]:
dados_ab.head()

,group,time,clickedTrue
0,A,2016-06-02 02:17:53,0
1,A,2016-06-02 03:03:54,0
2,A,2016-06-02 03:18:56,1
3,B,2016-06-02 03:23:43,0
4,A,2016-06-02 04:04:00,0


In [3]:
dados_nps.head()

,id,response_status,how_long_listening,age,nps_score,gender
0,11706300,Complete,Less than 6 months,25-34,10.0,Female
1,11706302,Complete,1 year to less than 3 years,25-34,10.0,Female
2,11706307,Complete,6 months to less than a year,35-44,10.0,Female
3,11706312,Complete,Less than 6 months,35-44,10.0,Female
4,11706316,Complete,6 months to less than a year,25-34,10.0,Male


In [4]:
# Estatísticas por grupo
statistics = dados_ab.groupby('group').agg(
    media_cliques = pd.NamedAgg(column='clickedTrue', aggfunc='mean'),
    dp_cliques = pd.NamedAgg(column='clickedTrue', aggfunc='std'),
    n = pd.NamedAgg(column='clickedTrue', aggfunc='count')
)

# Exibir as estatísticas
print(statistics)

       media_cliques  dp_cliques    n
group                                
A               0.04    0.196155  500
B               0.08    0.271565  500


In [5]:
# Dividir os grupos
grA = dados_ab[dados_ab['group'] == 'A']['clickedTrue']
grB = dados_ab[dados_ab['group'] == 'B']['clickedTrue']

# Aplicar o teste t de Student para duas amostras independentes
result = st.ttest_ind(a=grA, b=grB, alternative='two-sided')
print(result)

Ttest_indResult(statistic=-2.669938469060931, pvalue=0.007709783987515947)


In [6]:
"""
O teste t de Student, usado no contexto de st.ttest_ind no SciPy, é um método estatístico para determinar se há diferença significativa entre as médias de duas amostras independentes.
Especificamente, ele avalia se a diferença entre as médias das duas amostras é estatisticamente significativa ou se pode ter ocorrido devido a flutuações aleatórias.

Entendendo os resultados:

O resultado do teste Ttest_indResult contém duas estatísticas principais:

Statistic (Estatística): O valor da estatística de teste (no seu caso, -2.669938469060931).
Essa estatística de teste é o valor calculado que ajuda a determinar se há diferença entre as médias das amostras. No teste t de Student, quanto maior (ou menor, dependendo do teste de uma ou duas caudas) for o valor absoluto da estatística, maior a evidência contra a hipótese nula.

P-value (Valor p): O p-valor (no seu caso, 0.007709783987515948) é a probabilidade de observar uma diferença igual ou mais extrema entre as médias das amostras, supondo que a hipótese nula (as médias são iguais) seja verdadeira.
Um p-valor baixo indica que a diferença observada nas médias das amostras é menos provável devido ao acaso quando a hipótese nula é verdadeira.
Normalmente, um p-valor menor que um nível de significância (geralmente 0,05) sugere evidência suficiente para rejeitar a hipótese nula.

Conclusões possíveis:

Estatística (Statistic): O valor negativo sugere que a média do grupo 'A' é menor do que a média do grupo 'B'.
No entanto, para o teste t de Student, o foco principal é o valor absoluto da estatística para avaliar a diferença, não necessariamente a direção dessa diferença.

P-value (Valor p): O p-valor é 0.0077, que é menor do que o nível de significância comum de 0.05. Isso sugere que há evidências estatísticas suficientes para rejeitar a hipótese nula de que não há diferença entre as médias dos grupos.
Portanto, pode-se concluir que há uma diferença estatisticamente significativa entre as médias das amostras 'A' e 'B'.

Portanto, com base nos resultados obtidos, pode-se concluir que existe uma diferença estatisticamente significativa entre as médias dos grupos 'A' e 'B'.
"""

"\nO teste t de Student, usado no contexto de st.ttest_ind no SciPy, é um método estatístico para determinar se há diferença significativa entre as médias de duas amostras independentes.\nEspecificamente, ele avalia se a diferença entre as médias das duas amostras é estatisticamente significativa ou se pode ter ocorrido devido a flutuações aleatórias.\n\nEntendendo os resultados:\n\nO resultado do teste Ttest_indResult contém duas estatísticas principais:\n\nStatistic (Estatística): O valor da estatística de teste (no seu caso, -2.669938469060931).\nEssa estatística de teste é o valor calculado que ajuda a determinar se há diferença entre as médias das amostras. No teste t de Student, quanto maior (ou menor, dependendo do teste de uma ou duas caudas) for o valor absoluto da estatística, maior a evidência contra a hipótese nula.\n\nP-value (Valor p): O p-valor (no seu caso, 0.007709783987515948) é a probabilidade de observar uma diferença igual ou mais extrema entre as médias das amostr

In [7]:
## Mesmo teste mas com outra biblioteca

# Teste t usando a biblioteca Pingouin
pg.ttest(x=grA, y=grB, alternative='two-sided', confidence=0.95)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.669938,998,two-sided,0.00771,"[-0.07, -0.01]",0.168862,2.349,0.760344


In [8]:
'''
.size(): Após o agrupamento, o método .size() conta o número de ocorrências em cada grupo. Ele conta a quantidade de registros em cada grupo formado pelos valores únicos presentes na coluna 'response_status'.

.to_frame('n'): O método .to_frame('n') converte a série resultante (com os tamanhos dos grupos) em um DataFrame, e renomeia a coluna resultante como 'n'. Isso cria uma estrutura tabular com uma coluna indicando os diferentes valores de 'response_status' e outra coluna ('n') contendo o número de ocorrências de cada valor.

.reset_index(): O método .reset_index() redefinição o índice do DataFrame para um padrão sequencial. Ele transforma os índices de grupo (criados pelo groupby) de volta em colunas, criando um índice numérico padrão, iniciando do zero, e o nomeia como uma coluna.
'''

#Perfil das Respostas
dados_nps.groupby('response_status') .size() .to_frame('n') .reset_index()

,response_status,n
0,Complete,2281
1,Incomplete,265
2,Terminated,33


In [9]:
dados_nps[dados_nps['nps_score'].isnull()]

,id,response_status,how_long_listening,age,nps_score,gender
17,11706467,Incomplete,Less than 6 months,18-24,NaN,NaN
31,11706938,Incomplete,1 year to less than 3 years,25-34,NaN,NaN
32,11706979,Incomplete,6 months to less than a year,25-34,NaN,NaN
43,11707426,Incomplete,6 months to less than a year,25-34,NaN,NaN
48,11707719,Incomplete,3 years to less than 5 years,35-44,NaN,NaN
...,...,...,...,...,...,...
2546,13093216,Incomplete,6 months to less than a year,35-44,NaN,NaN
2556,13278063,Incomplete,3 years to less than 5 years,18-24,NaN,NaN
2570,13565327,Complete,1 year to less than 3 years,45-54,NaN,Female
2572,13601847,Incomplete,3 years to less than 5 years,25-34,NaN,NaN


In [10]:
#Limpeza dos Dados
dados_nps_filtrados = dados_nps[(dados_nps['response_status'] == 'Complete') & (dados_nps['nps_score'].notna())] 

In [11]:
#Fazendo a média, std deviation e size da coluna nps_score. Tudo isso agrupado por age, que fizemos através do groupby no início
dados_nps_filtrados.groupby('age').agg(
    media_nps = pd.NamedAgg(column='nps_score', aggfunc='mean'),
    dp_nps = pd.NamedAgg(column='nps_score', aggfunc='std'),
    n = pd.NamedAgg(column='nps_score', aggfunc='size')
).reset_index()

,age,media_nps,dp_nps,n
0,18-24,9.464539,1.116275,282
1,25-34,9.694828,0.957639,580
2,35-44,9.707612,0.979501,578
3,45-54,9.719039,0.928254,541
4,55-64,9.733871,0.923020,248
5,65-74,9.423077,1.361560,26
6,75+,8.000000,0.000000,2


In [12]:
#Filtrando 75+
dados_nps_filtrados_aj = dados_nps_filtrados[dados_nps_filtrados['age'] != '75+']

In [13]:
#Teste de Hipóteses

#H0 = Todas as faixas etárias apresentam a mesma média de NPS

#H1 = Alguma das faixas etárias apresenta média de NPS diferente das demais

#Rejeitar Hip Nula quando p-value for menor que 0,05.

# Separar os grupos de acordo com as faixas etárias
dados_18_24 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '18-24']['nps_score']
dados_25_34 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '25-34']['nps_score']
dados_35_44 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '35-44']['nps_score']
dados_45_54 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '45-54']['nps_score']
dados_55_64 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '55-64']['nps_score']
dados_65_74 = dados_nps_filtrados_aj[dados_nps_filtrados_aj['age'] == '65-74']['nps_score']

# Realizar o teste ANOVA
st.f_oneway(dados_18_24, dados_25_34, dados_35_44, dados_45_54, dados_55_64, dados_65_74)


F_onewayResult(statistic=3.5221660981040768, pvalue=0.0035606861304276695)

In [14]:
#Mesmo teste agora com a biblioteca
pg.anova(dv='nps_score', between='age', data=dados_nps_filtrados_aj, detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,age,16.888794,5,3.377759,3.522166,0.003561,0.00777
1,Within,2156.791916,2249,0.959000,NaN,NaN,NaN


In [23]:
#Exercicio 1

#Limpeza dos Dados
dados_nps_filtrados_ex1 = dados_nps[(dados_nps['response_status'] == 'Complete') & (dados_nps['nps_score'].notna()) & (dados_nps['gender'].notna())]

dados_nps_filtrados_ex1

,id,response_status,how_long_listening,age,nps_score,gender
0,11706300,Complete,Less than 6 months,25-34,10.0,Female
1,11706302,Complete,1 year to less than 3 years,25-34,10.0,Female
2,11706307,Complete,6 months to less than a year,35-44,10.0,Female
3,11706312,Complete,Less than 6 months,35-44,10.0,Female
4,11706316,Complete,6 months to less than a year,25-34,10.0,Male
...,...,...,...,...,...,...
2573,13610170,Complete,6 months to less than a year,25-34,10.0,Female
2574,13640772,Complete,3 years to less than 5 years,18-24,10.0,Female
2576,13732056,Complete,1 year to less than 3 years,18-24,10.0,Female
2577,13734055,Complete,Less than 6 months,25-34,10.0,Male


In [16]:
dados_nps_filtrados_ex1.groupby('gender').agg(
    media_nps = pd.NamedAgg(column='nps_score', aggfunc='mean'),
    dp_nps = pd.NamedAgg(column='nps_score', aggfunc='std'),
    n = pd.NamedAgg(column='nps_score', aggfunc='size')
).reset_index()

,gender,media_nps,dp_nps,n
0,Female,9.744352,0.865792,1682
1,Male,9.477718,1.204138,561


In [20]:
#Comparando NPS dentre os genders
pg.anova(dv='nps_score', between='gender', data=dados_nps_filtrados_ex1, detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,gender,29.908135,1,29.908135,32.346885,1.457979e-08,0.014229
1,Within,2072.042823,2241,0.924606,NaN,NaN,NaN


In [26]:
# Dividir os grupos
grA = dados_nps_filtrados_ex1[dados_nps_filtrados_ex1['gender'] == 'Male']['nps_score']
grB = dados_nps_filtrados_ex1[dados_nps_filtrados_ex1['gender'] == 'Female']['nps_score']

# Aplicar o teste t de Student para duas amostras independentes
result = st.ttest_ind(a=grA, b=grB, alternative='two-sided')
print(result)

Ttest_indResult(statistic=-5.687432219135097, pvalue=1.4579787743425994e-08)


In [27]:
# Teste t usando a biblioteca Pingouin
pg.ttest(x=grA, y=grB, alternative='two-sided', confidence=0.95)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.843693,762.2213,two-sided,0.000002,"[-0.37, -0.16]",0.277291,5805.253,0.999902
